In [76]:
import copy
import random

In [77]:
class Environment:
    def __init__(self, n):
        self.n = n
        self.state = [[n * i + j + 1 for j in range(n)] for i in range(n)]
        self.state[n - 1][n - 1] = 0
        self.state_string = self.get_state_string(self.state)
        self.agent_pos = [n - 1, n - 1]
        self.terminal_state_string = self.state_string


    def set_state(self, state):
        
        for i in range(self.n):
            for j in range(self.n):
                self.state[i][j] = int(state[2 * self.n * i + j * 2: 2 * self.n * i + j * 2 + 2])

                if self.state[i][j] == 0:
                    self.agent_pos = [i, j]

        self.state_string = self.get_state_string(self.state)


    def get_actions(self):
        return [0, 1, 2, 3]
    

    # 0: up, 1: right, 2: down, 3: left
    # Returns next state and agent's next position
    def get_next_state(self, action):
        
        if action == 0:
            if self.agent_pos[0] == 0:
                return self.state, self.agent_pos
            else:
                result = copy.deepcopy(self.state)
                result[self.agent_pos[0]][self.agent_pos[1]] = result[self.agent_pos[0] - 1][self.agent_pos[1]]
                result[self.agent_pos[0] - 1][self.agent_pos[1]] = 0
                return result, [self.agent_pos[0] - 1, self.agent_pos[1]]

        elif action == 1:
            if self.agent_pos[1] == self.n - 1:
                return self.state, self.agent_pos
            else:
                result = copy.deepcopy(self.state)
                result[self.agent_pos[0]][self.agent_pos[1]] = result[self.agent_pos[0]][self.agent_pos[1] + 1]
                result[self.agent_pos[0]][self.agent_pos[1] + 1] = 0
                return result, [self.agent_pos[0], self.agent_pos[1] + 1]

        elif action == 2:
            if self.agent_pos[0] == self.n - 1:
                return self.state, self.agent_pos
            else:
                result = copy.deepcopy(self.state)
                result[self.agent_pos[0]][self.agent_pos[1]] = result[self.agent_pos[0] + 1][self.agent_pos[1]]
                result[self.agent_pos[0] + 1][self.agent_pos[1]] = 0
                return result, [self.agent_pos[0] + 1, self.agent_pos[1]]

        elif action == 3:
            if self.agent_pos[1] == 0:
                return self.state, self.agent_pos
            else:
                result = copy.deepcopy(self.state)
                result[self.agent_pos[0]][self.agent_pos[1]] = result[self.agent_pos[0]][self.agent_pos[1] - 1]
                result[self.agent_pos[0]][self.agent_pos[1] - 1] = 0
                return result, [self.agent_pos[0], self.agent_pos[1] - 1]


    def get_state_string(self, state):
        result = ""
        for i in range(self.n):
            for j in range(self.n):
                tmp = str(state[i][j])
                if len(tmp) == 1:
                    result += "0" + tmp
                else:
                    result += tmp
        return result
                

    def get_reward(self, state, next_state, state_string, next_state_string):
        if next_state_string == self.terminal_state_string:
            R = 0
        else:
            score = self.complete_row_column_score(next_state)
            R = score + 2 - self.n
        return self.potential(next_state) - self.potential(state) + R
    

    def potential(self, state):
        result = 0

        h1 = 0
        for i in range(self.n):
            for j in range(self.n):
                if state[i][j] == 0:
                    i2 = self.n - 1
                    j2 = self.n - 1
                else:
                    i2 = (state[i][j] - 1) // self.n
                    j2 = state[i][j] - i2 * self.n - 1
                
                h1 += abs(i2 - i) + abs(j2 - j)

        # h1 = -1.0 * h1 / (2 * (self.n * self.n) * (self.n - 1))
        h1 = -1.0 * h1 / (self.n * self.n)

        x = (1 + self.n) * self.n / 2

        rows = self.complete_row_score(state)
        columns = self.complete_column_score(state)

        # h2 = ((rows - x) + (columns - x)) / (2 * x)
        h2 = ((rows - x) + (columns - x))

        row_col = self.complete_row_column_score(state)

        # h3 = (row_col / (x - 6) - 1) if self.n > 3 else 0

        # result = 4.0 * (h1 + h2 + h3) / 3.0

        result = h1 + h2

        

        # if row_col >= 4:
        #     print("row col score: {}".format(row_col))
        
                
        return result
    

    def complete_row_score(self, state):
        result = 0
        for i in range(self.n):
            in_order = 0
            for j in range(self.n):
                if state[i][j] == i * self.n + j + 1:
                    in_order += 1
            if in_order == self.n:
                result += self.n - i
                    
        return result
    

    def complete_column_score(self, state):
        result = 0
        
        for j in range(self.n):
            in_order = 0
            for i in range(self.n):
                if state[i][j] == i * self.n + j + 1:
                    in_order += 1
            if in_order == self.n:
                result += self.n - j
        
        return result
    

    def complete_row_column_score(self, state):
        result = 0
        for i in range(self.n - 3):
            in_order_row = 0
            in_order_column = 0
            for j in range(self.n):
                if state[i][j] == i * self.n + j + 1:
                    in_order_row += 1
                if state[i][j] != i * self.n + j + 1:
                    return result
                if state[j][i] == j * self.n + i + 1:
                    in_order_column += 1
                if state[j][i] != j * self.n + i + 1:
                    return result
                
            if in_order_row == self.n and in_order_column == self.n:
                result += 1

        return result
    

    def is_valid(self, state):
        if self.n % 2 == 1:
            if self.inversion_count(state) % 2 == 0:
                return True
            else:
                return False
        else:
            if (self.inversion_count(state) % 2 == 1) and ((self.n - self.agent_pos[0]) % 2 == 0):
                return True
            elif (self.inversion_count(state) % 2 == 0) and ((self.n - self.agent_pos[0]) % 2 == 1):
                return True
            else:
                return False
            

    def inversion_count(self, state):
        result = 0
        lst = []
        tmp_state = [[0 for i in range(self.n)] for j in range(self.n)]

        for i in range(self.n):
            for j in range(self.n):
                tmp_state[i][j] = int(state[2 * self.n * i + j * 2: 2 * self.n * i + j * 2 + 2])


        for i in range(self.n):
            for j in range(self.n):
                if tmp_state[i][j] == 0:
                    continue
                lst.append(tmp_state[i][j])

        for i in range(len(lst)):
            for j in range(i + 1, len(lst)):
                if lst[i] > lst[j]:
                    result += 1

        return result
    

    def print_state(self):
        for i in range(self.n):
            for j in range(self.n):
                print(self.state[i][j], end = " ")
            print()

In [78]:
class Agent:
    def __init__(self, env, V = None, S0 = None):
        self.env = env
        self.V = V
        self.S0 = S0


    def async_value_iteration(self, S0, max_episodes = 100, max_steps = None , learning_rate = 1.0, epsilon_start = 0.3, epsilon_end = 0.05):
        
        if not self.check_validity_from_string(S0):
            return
        
        if self.V is None:
            self.V = dict()
        
        self.S0 = S0

        if max_steps is None:
            max_steps = 4 * (self.env.n ** 3)
        
        epsilon = epsilon_start
        epsilon_coeff = (epsilon_end / epsilon_start) ** (1 / max_episodes)
        episode = 0

        while episode < max_episodes:
            
            self.env.set_state(S0)
            step = 0
            done = False
            while True:
                max_G, next_S, next_S_string, next_agent_pos = self.select_action(epsilon)
                
                if self.env.state_string not in self.V:
                    self.V[self.env.state_string] = 0
                
                self.V[self.env.state_string] = self.V[self.env.state_string] + learning_rate * (max_G - self.V[self.env.state_string])
                
                self.env.state = next_S
                self.env.state_string = next_S_string
                self.env.agent_pos = next_agent_pos

                step += 1
                
                if self.env.state_string == self.env.terminal_state_string:
                    done = True

                if self.env.state_string == self.env.terminal_state_string or step >= max_steps:
                    epsilon *= epsilon_coeff
                    break
            
            episode += 1
            
            if done:
                print("------ Episode {} successful ------".format(episode))
            
            if episode % (max_episodes // 10) == 0:
                print("Episode: {}".format(episode))


    # Returns maximum expected return from the current state
    def select_action(self, epsilon):
        
        results = []

        max_return = -1e9
        max_action = None
        for action in self.env.get_actions():

            nextS, next_agent_pos = self.env.get_next_state(action)
            nextS_index = self.env.get_state_string(nextS)
            
            if nextS_index not in self.V:
                nextS_value = 0
            else:
                nextS_value = self.V[nextS_index]

            expected_return = self.env.get_reward(self.env.state, nextS, self.env.state_string, nextS_index) + nextS_value

            results.append((expected_return, nextS, nextS_index, next_agent_pos))
        
            if expected_return > max_return:
                max_return = expected_return
                max_action = action
    
        if random.random() > epsilon:
            action = max_action
        else:
            action = random.choice(self.env.get_actions())

        return max_return, results[action][1], results[action][2], results[action][3]
    

    def async_value_iteration_with_stack(self, S0, max_episodes = 100, max_steps = None , learning_rate = 1.0, epsilon_start = 0.3, epsilon_end = 0.05):
        
        if not self.check_validity_from_string(S0):
            return
        
        if self.V is None:
            self.V = dict()
        
        self.S0 = S0

        if max_steps is None:
            max_steps = 4 * (self.env.n ** 3)

        epsilon = epsilon_start
        epsilon_coeff = (epsilon_end / epsilon_start) ** (1 / max_episodes)
        episode = 0

        while episode < max_episodes:
            stack = []
            
            self.env.set_state(S0)

            stack.append((self.env.state, self.env.state_string, self.env.agent_pos))

            step = 0
            done = False

            while True:
                max_G, next_S, next_S_string, next_agent_pos = self.select_action(epsilon)

                if self.env.state_string not in self.V:
                    self.V[self.env.state_string] = 0
                
                self.V[self.env.state_string] = self.V[self.env.state_string] + learning_rate * (max_G - self.V[self.env.state_string])

                stack.append((next_S, next_S_string, next_agent_pos))

                self.env.state = next_S
                self.env.state_string = next_S_string
                self.env.agent_pos = next_agent_pos


                step += 1


                if self.env.state_string == self.env.terminal_state_string:
                    done = True
                
                if self.env.state_string == self.env.terminal_state_string or step >= max_steps:
                    epsilon *= epsilon_coeff
                    stack.pop()

                    while len(stack) > 0:
                        self.env.state, self.env.state_string, self.env.agent_pos = stack.pop()

                        max_G, _, _, _ = self.select_action(epsilon)
                        
                        if self.env.state_string not in self.V:
                            self.V[self.env.state_string] = 0
                
                        self.V[self.env.state_string] = self.V[self.env.state_string] + learning_rate * (max_G - self.V[self.env.state_string])

                    break

                # if step >= max_steps:
                #     break

            episode += 1
            
            if done:
                print("------ Episode {} successful ------".format(episode))
            
            if episode % (max_episodes // 20) == 0:
                print("Episode: {}".format(episode))
    

    def check_validity_from_string(self, state_string):
        if self.env.is_valid(state_string) == True:
            return True
        else:
            print("State is not valid!")
            return False


    def exploit(self, S0 = None):
        
        if S0 is None:
            S0 = self.S0

        if not self.check_validity_from_string(S0):
            return

        self.env.set_state(S0)
        
        step = 0

        self.env.print_state()

        while self.env.state_string != self.env.terminal_state_string:
            step += 1
            _, next_S, next_S_string, next_agent_pos = self.select_action(0.0)
            
            self.env.state = next_S
            self.env.state_string = next_S_string
            self.env.agent_pos = next_agent_pos
            
            print("- step: {} -".format(step))
            self.env.print_state()

            if step > 4 * (self.env.n ** 3):
                break

In [79]:
def get_random_state_string(n):
    
    lst = list(range(n * n))
    random.shuffle(lst)

    result = ""

    for i in range(n * n):
        tmp = str(lst[i])

        if len(tmp) == 1:
            result += "0" + tmp
        else:
            result += tmp

    return result

In [81]:
n = 4
env = Environment(n)

agent = Agent(env)

initial_state_string = get_random_state_string(n)

agent.async_value_iteration_with_stack(initial_state_string, max_episodes = 40000, max_steps = 500, learning_rate = 1.0, epsilon_start = 0.2, epsilon_end = 0.05)

Episode: 2000
Episode: 4000


KeyboardInterrupt: 

In [35]:
agent.V[initial_state_string]

-19.999999999999993

In [33]:
agent.exploit()

8 4 6 
7 1 3 
5 2 0 
- step: 1 -
8 4 6 
7 1 0 
5 2 3 
- step: 2 -
8 4 6 
7 0 1 
5 2 3 
- step: 3 -
8 4 6 
0 7 1 
5 2 3 
- step: 4 -
8 4 6 
5 7 1 
0 2 3 
- step: 5 -
8 4 6 
5 7 1 
2 0 3 
- step: 6 -
8 4 6 
5 0 1 
2 7 3 
- step: 7 -
8 4 6 
5 1 0 
2 7 3 
- step: 8 -
8 4 0 
5 1 6 
2 7 3 
- step: 9 -
8 0 4 
5 1 6 
2 7 3 
- step: 10 -
0 8 4 
5 1 6 
2 7 3 
- step: 11 -
5 8 4 
0 1 6 
2 7 3 
- step: 12 -
5 8 4 
2 1 6 
0 7 3 
- step: 13 -
5 8 4 
2 1 6 
7 0 3 
- step: 14 -
5 8 4 
2 1 6 
7 0 3 
- step: 15 -
5 8 4 
2 1 6 
7 0 3 
- step: 16 -
5 8 4 
2 1 6 
7 0 3 
- step: 17 -
5 8 4 
2 1 6 
7 0 3 
- step: 18 -
5 8 4 
2 1 6 
7 0 3 
- step: 19 -
5 8 4 
2 1 6 
7 0 3 
- step: 20 -
5 8 4 
2 1 6 
7 0 3 
- step: 21 -
5 8 4 
2 1 6 
7 0 3 
- step: 22 -
5 8 4 
2 1 6 
7 0 3 
- step: 23 -
5 8 4 
2 1 6 
7 0 3 
- step: 24 -
5 8 4 
2 1 6 
7 0 3 
- step: 25 -
5 8 4 
2 1 6 
7 0 3 
- step: 26 -
5 8 4 
2 1 6 
7 0 3 
- step: 27 -
5 8 4 
2 1 6 
7 0 3 
- step: 28 -
5 8 4 
2 1 6 
7 0 3 
- step: 29 -
5 8 4 
2 1 6 
7 0 3 
- 

In [14]:
agent.exploit(get_random_state_string(3))

6 3 1 
8 4 2 
0 5 7 
- step: 1 -
6 3 1 
0 4 2 
8 5 7 
- step: 2 -
6 3 1 
4 0 2 
8 5 7 
- step: 3 -
6 3 1 
4 2 0 
8 5 7 
- step: 4 -
6 3 1 
4 2 0 
8 5 7 
- step: 5 -
6 3 1 
4 2 0 
8 5 7 
- step: 6 -
6 3 1 
4 2 0 
8 5 7 
- step: 7 -
6 3 1 
4 2 0 
8 5 7 
- step: 8 -
6 3 1 
4 2 0 
8 5 7 
- step: 9 -
6 3 1 
4 2 0 
8 5 7 
- step: 10 -
6 3 1 
4 2 0 
8 5 7 
- step: 11 -
6 3 1 
4 2 0 
8 5 7 
- step: 12 -
6 3 1 
4 2 0 
8 5 7 
- step: 13 -
6 3 1 
4 2 0 
8 5 7 
- step: 14 -
6 3 1 
4 2 0 
8 5 7 
- step: 15 -
6 3 1 
4 2 0 
8 5 7 
- step: 16 -
6 3 1 
4 2 0 
8 5 7 
- step: 17 -
6 3 1 
4 2 0 
8 5 7 
- step: 18 -
6 3 1 
4 2 0 
8 5 7 
- step: 19 -
6 3 1 
4 2 0 
8 5 7 
- step: 20 -
6 3 1 
4 2 0 
8 5 7 
- step: 21 -
6 3 1 
4 2 0 
8 5 7 
- step: 22 -
6 3 1 
4 2 0 
8 5 7 
- step: 23 -
6 3 1 
4 2 0 
8 5 7 
- step: 24 -
6 3 1 
4 2 0 
8 5 7 
- step: 25 -
6 3 1 
4 2 0 
8 5 7 
- step: 26 -
6 3 1 
4 2 0 
8 5 7 
- step: 27 -
6 3 1 
4 2 0 
8 5 7 
- step: 28 -
6 3 1 
4 2 0 
8 5 7 
- step: 29 -
6 3 1 
4 2 0 
8 5 7 
- 